In [23]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [24]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [26]:
train_x= train_df.drop(columns=['PRODUCT_ID','Y_Quality', 'Y_Class'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID'])

In [27]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [28]:
from sklearn.model_selection import train_test_split

# train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=34)

In [29]:
X_train = X_train.fillna(0)
X_valid = X_valid.fillna(0)
test_x = test_x.fillna(0)

In [30]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [31]:
# 각각 다른 분류 모델을 생성합니다.
cat = CatBoostClassifier(max_depth = 9, random_state=42)
knn = KNeighborsClassifier()
svm = SVC(probability=True, random_state=42)

In [32]:
# 각 모델을 앙상블 투표로 결합합니다.
voting_clf = VotingClassifier(
    estimators=[('cat', cat), ('knn', knn), ('svm', svm)],
    voting='soft'
)

In [33]:
# 앙상블 투표 모델을 학습합니다.
voting_clf.fit(train_x, train_y)

Learning rate set to 0.079629
0:	learn: 1.0452154	total: 1.18s	remaining: 19m 43s
1:	learn: 1.0021327	total: 1.68s	remaining: 13m 58s
2:	learn: 0.9649844	total: 2.32s	remaining: 12m 52s
3:	learn: 0.9359820	total: 3.13s	remaining: 12m 59s
4:	learn: 0.9099995	total: 4s	remaining: 13m 16s
5:	learn: 0.8881819	total: 4.84s	remaining: 13m 21s
6:	learn: 0.8681146	total: 5.68s	remaining: 13m 25s
7:	learn: 0.8497965	total: 6.45s	remaining: 13m 20s
8:	learn: 0.8348768	total: 7.03s	remaining: 12m 53s
9:	learn: 0.8218836	total: 7.53s	remaining: 12m 24s
10:	learn: 0.8093706	total: 8.01s	remaining: 11m 59s
11:	learn: 0.7961317	total: 8.5s	remaining: 11m 39s
12:	learn: 0.7861521	total: 8.98s	remaining: 11m 21s
13:	learn: 0.7767459	total: 9.49s	remaining: 11m 8s
14:	learn: 0.7692361	total: 9.96s	remaining: 10m 54s
15:	learn: 0.7629916	total: 10.5s	remaining: 10m 43s
16:	learn: 0.7559807	total: 10.9s	remaining: 10m 32s
17:	learn: 0.7472701	total: 11.5s	remaining: 10m 25s
18:	learn: 0.7426909	total: 11.

VotingClassifier(estimators=[('cat',
                              <catboost.core.CatBoostClassifier object at 0x7f9fb98961f0>),
                             ('knn', KNeighborsClassifier()),
                             ('svm', SVC(probability=True, random_state=42))],
                 voting='soft')

In [34]:
# 검증용 데이터셋을 사용하여 앙상블 투표 모델의 정확도를 측정합니다.
y_pred = voting_clf.predict(test_x)
#accuracy = accuracy_score(y_test, y_pred)
#print(f"앙상블 투표 모델의 정확도: {accuracy:.4f}")

In [35]:
submit1 = pd.read_csv('/content/sample_submission.csv')
submit1['Y_Class'] = y_pred

In [39]:
submit1.to_csv('sub.csv',index=False)

In [22]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 5.6 MB/s eta 0:00:00
